In [ ]:
import numpy as np
import pandas as pd
import regex as re
from bs4 import BeautifulSoup
import contractions
import unidecode
import re
import pickle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
import string
from sklearn.utils import shuffle
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification,RobertaForSequenceClassification,RobertaTokenizer
from transformers import EarlyStoppingCallback
# from gensim.parsing.preprocessing import remove_stopwords
from transformers import XLMRobertaModel, XLMRobertaTokenizer
import os


In [ ]:

def removeHtml(sentence):
    soup = BeautifulSoup(sentence, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

def removePuncu(sentence):
    res = re.sub(r'[^\w\s]', '', sentence)
    return res

def removeLink(sentence):
    result = re.sub(r'http\S+', '', sentence)
    return result

def removeWhiteSpaces(sentence):
    sentence = sentence.strip()
    return " ".join(sentence.split())

def expand_contractions(sentence):
    sentence = contractions.fix(sentence)
    return sentence

def removeAccented(sentence):
    sentence = unidecode.unidecode(sentence)
    return sentence

def removeSpec(sentence):
    sentence = re.sub("[^A-Z]", " ", sentence, 0, re.IGNORECASE)
    return sentence

def removeNum(sentence):
    pattern = r'[0-9]'
    sentence = re.sub(pattern, '', sentence)
    return sentence

def lowerCase(sentence):
    sentence = sentence.lower()
    return sentence
    
# def removeUser(sentence):
#     remove = re.compile('(\s*)user(\s*)')
#     sentence = remove.sub(" ", sentence)
#     return sentence

# with open(r'C:\Users\LEGION\Documents\Untitled Folder 2\Emoji_Dict.p', 'rb') as fp:
#     Emoji_Dict = pickle.load(fp)
# Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

# def convertEmoji(sentence):
#     for emot in Emoji_Dict:
#         sentence = re.sub(r'('+emot+')', "".join(Emoji_Dict[emot].replace(",","").replace(":","").split()),sentence)
#     return sentence

def add_labels(value):
    if value == 'positive':
        return 2
    elif value == 'negative':
        return 0
    elif value == 'neutral':
        return 1
    

In [ ]:
def preprocessing(data):
    sent_or_word = 'translation_sentence_level'
    data[sent_or_word] = data[sent_or_word].apply(lambda x: lowerCase(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removePuncu(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeSpec(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeHtml(x))

    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeAccented(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeLink(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeNum(x))

    data[sent_or_word] = data[sent_or_word].apply(lambda x: expand_contractions(x))
#     da[sent_or_word] = da[sent_or_word].apply(lambda x: convertEmoji(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeWhiteSpaces(x))
    # data[sent_or_word] = data[sent_or_word].apply(lambda x: removeUser(x))
    x = data[sent_or_word].to_list()
    y = list(data['categorical_labels']) 
    
    return x,y


In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
model_results_dict = {"Model_Name":[], "Accuracy": [], 'Recall': [], 'Precision': [], "f1":[]}

In [11]:
path = "modelMergedRemoveEmojiRoberta/roberta-base_Train_merged_Roberta_Removed_Emoji"
max_length = 128
num_labels = 3
# task = 'sentiment'
modelName = "roberta-base"
# modelName = f"cardiffnlp/twitter-roberta-base-{task}"
# safe_model_name = sanitize(model_name)

tokenizer = AutoTokenizer.from_pretrained(modelName)

model = AutoModelForSequenceClassification.from_pretrained(path)


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /home/ablstation2/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 1,

In [12]:
newModelNameForSaving = modelName.split("/")[-1]


In [13]:
allTest = os.listdir('emojiRemovedFiles/test')

for file in allTest:
    newModelNameForSaving = modelName.split("/")[-1]
    def compute_metrics(p):
        pred, labels = p
        pred = np.argmax(pred, axis=1)
        accuracy = accuracy_score(y_true=labels, y_pred=pred)
        recall = recall_score(y_true=labels, y_pred=pred,average='weighted')
        precision = precision_score(y_true=labels, y_pred=pred,average='weighted')
        f1 = f1_score(y_true=labels, y_pred=pred,average='weighted')

        return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

    args = TrainingArguments(
        output_dir="checkpointsTestFileToFile/{}_{}".format(newModelNameForSaving,file.split(".")[0]),
        evaluation_strategy="epoch",
        per_device_train_batch_size=256,
        per_device_eval_batch_size=256,
        num_train_epochs=5,
        seed=7,
        load_best_model_at_end=True,
        save_strategy='epoch',
        overwrite_output_dir=True)
        #save_strategy = "no"

    trainer = Trainer(
      model=model,
      args=args,
      compute_metrics = compute_metrics,
      callbacks=[EarlyStoppingCallback(early_stopping_patience=3)])


    test_data = pd.read_csv("emojiRemovedFiles/test/{}".format(file),sep='\t')
    test_data = test_data[test_data['translation_sentence_level'].notna()]
    test_data['categorical_labels'] = test_data['label'].map(add_labels)

    X_test, y_test = preprocessing(test_data)


    X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=max_length,add_special_tokens=True)


    test_dataset = Dataset(X_test_tokenized, y_test)


    predictions = trainer.predict(test_dataset)

    pred = np.argmax(predictions.predictions, axis=-1)

  
    accuracy = accuracy_score(y_true=y_test, y_pred=pred)
    recall = recall_score(y_true=y_test, y_pred=pred,average='weighted')
    precision = precision_score(y_true=y_test, y_pred=pred,average='weighted')
    f1 = f1_score(y_true=y_test, y_pred=pred,average='weighted')

    print(modelName)
    print("accuracy: {}".format(accuracy))
    print("recall: {}".format(recall))
    print("precision: {}".format(precision))
    print("f1: {}".format(f1))

    model_results_dict['Model_Name'].append(file)
    model_results_dict['Accuracy'].append(accuracy)
    model_results_dict['Recall'].append(recall)
    model_results_dict['Precision'].append(precision)
    model_results_dict['f1'].append(f1)

    df = pd.DataFrame(data = model_results_dict)
    df.to_csv("{}_FileToFileRemovedEmojiRobertaresults.csv".format(newModelNameForSaving))

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
100%|██████████| 1/1 [10:28<00:00, 628.30s/it]
***** Running Prediction *****
  Num examples = 826
  Batch size = 256
/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
100%|██████████| 2/2 [00:00<00:00, 11.81it/s]PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should st

roberta-base
accuracy: 0.6101694915254238
recall: 0.6101694915254238
precision: 0.6566553530268914
f1: 0.6045043905764667


  0%|          | 0/1 [00:00<?, ?it/s]PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
100%|██████████| 1/1 [00:00<00:00, 32.32it/s]
***** Running Prediction *****
  Num examples = 453
  Batch size = 256
/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


roberta-base
accuracy: 0.41494845360824745
recall: 0.41494845360824745
precision: 0.545644292954916
f1: 0.4415317804974339


  0%|          | 0/1 [00:00<?, ?it/s]PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
100%|██████████| 1/1 [00:00<00:00, 33.41it/s]
***** Running Prediction *****
  Num examples = 1840
  Batch size = 256


...

roberta-base
accuracy: 0.618798955613577
recall: 0.618798955613577
precision: 0.6387907074004479
f1: 0.6194296783650607


/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
 67%|██████▋   | 2/3 [00:00<00:00, 12.67it/s]PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
100%|██████████| 3/3 [00:00<00:00,  9.11it/s]
***** Running Prediction *****
  Num examples = 203
  Batch size = 256
/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  wa

roberta-base
accuracy: 0.6158984635938544
recall: 0.6158984635938544
precision: 0.617179374247978
f1: 0.6160045326944797


  0%|          | 0/1 [00:00<?, ?it/s]

roberta-base
accuracy: 0.4039408866995074
recall: 0.4039408866995074
precision: 0.5575742821942243
f1: 0.4320450499016881
